In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

#Step 1: Data Extraction and Loading

#Extract Data: Load the necessary tables from the Ergast dataset, which includes drivers, constructors, races, qualifying, results, driverStandings, and constructorStandings.
#Load Data: Use a database client (e.g., MySQL, SQLite) or CSV files to load the data into a pandas DataFrame.
# Load data
constructors_df = pd.read_csv("f1db_csv/constructors.csv")
races_df=pd.read_csv("f1db_csv/races.csv")
quali_df=pd.read_csv("f1db_csv/qualifying.csv")
results_df=pd.read_csv("f1db_csv/results.csv")
driverStandings_df=pd.read_csv("f1db_csv/driver_standings.csv")
constructorStandings_df=pd.read_csv("f1db_csv/constructor_standings.csv")
drivers_df = pd.read_csv('f1db_csv/drivers.csv')

#Step 2: Data Preprocessing

#Merge Tables:
#Merge results with races to get race-specific details.
#Merge the combined DataFrame with qualifying to get qualifying positions.
#Merge with driverStandings and constructorStandings to include standings.
#Merge with drivers and constructors to add driver and constructor details.

# Merge data
data = results_df.merge(races_df, on="raceId")
data1 = data.merge(quali_df, on=["raceId","driverId","constructorId"], how="left")
data2 = data1.merge(driverStandings_df, on=["raceId","driverId"], how="left")
data3 = data2.merge(constructorStandings_df, on=['raceId','constructorId'])
data4 = data3.merge(drivers_df, on=['driverId'], how='left')
data5 = data4.merge(constructors_df, on=['constructorId'], how='left')

In [ ]:
#Handle Missing Values:
#Fill or drop missing values as appropriate for the analysis.
#Common strategies include filling with mean/median/mode or dropping rows with missing critical values.
#Convert Data Types:
#Ensure all columns are in the appropriate data type (e.g., integers for IDs, floats for numerical values).

columns=data5.columns

data5.replace('\\N', np.nan, inplace=True)
total_n_values = (data5.applymap(lambda x: x == '\\N')).sum().sum()
print(total_n_values)

data5.to_csv('output.csv', index=False)
f1data=pd.read_csv("output.csv", low_memory=False, index_col=False)
f1data.drop(['drivers_url','const_url'], axis=1, inplace=True)

In [ ]:
# Sorting the 'year' column in ascending order
# Assuming the 'year' column exists in the dataframe
if 'year' in df.columns:
    sorted_years = data5['year'].sort_values(ascending=True)
    sorted_years.reset_index(drop=True, inplace=True)
else:
    sorted_years = 'The column "year" does not exist in the dataframe.'

import ace_tools as tools; tools.display_dataframe_to_user(name="Sorted Year Column", dataframe=sorted_years)

sorted_years

In [ ]:
#Step 3: Feature Engineering

#Create New Features:
#Qualifying Position: Use the qualifying position as a feature.
#Previous Race Results: Calculate the average finishing position of the driver and constructor in previous races.
#Circuit Characteristics: Extract features such as circuit length, number of turns, and altitude.
#Weather Conditions: If available, include weather conditions like temperature, rain probability, etc.
#Driver and Constructor Standings: Use the current standings points and positions.
#Normalize/Standardize Features: Scale numerical features to ensure they have a mean of 0 and standard deviation of 1 (standardization) or scale between 0 and 1 (normalization).

# Feature engineering
f1data['average_finish'] = f1data.groupby('driverId')['results_positionOrder'].transform('mean')
f1data['date']=pd.to_datetime(f1data['races_date']).apply(lambda x: x.year)
f1data['age'] = f1data['date'] - pd.to_datetime(f1data['drivers_dob']).dt.year
f1data['experience'] = f1data['year'] - f1data.groupby('driverId')['year'].transform('min')
f1data['average_lap_time'] = f1data.groupby(['raceId', 'driverId'])['results_milliseconds'].transform('mean')

# Select features and target
features = ['age','experience','results_grid','quali_position','average_lap_time','average_finish','driver_standings_points','driver_standings_position','const_standing_points','const_standing_position']
target = 'results_positionOrder'
X = f1data[features]
y = f1data[target]

# Handling missing values
X.fillna(X.mean(), inplace=True)

# Encoding categorical variables
X = pd.get_dummies(X)

In [ ]:
#Step 4: Data Splitting

#Split the Data:
#Split the data into training and test sets (e.g., 80% training, 20% testing).
#Optionally, create a validation set from the training set for hyperparameter tuning.

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale features
# StandardScaler
#scaler = StandardScaler()
#X_train = scaler.fit_transform(X_train)
#X_test = scaler.transform(X_test)

In [ ]:
#XGBoost model
# Define the model
xgb_model = XGBRegressor(objective='reg:squarederror', n_estimators=100, max_depth=5, learning_rate=0.1, subsample=0.8, random_state=42)

# Train the model
xgb_model.fit(X_train, y_train)

# Make predictions
y_pred = xgb_model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f'Mean Squared Error: {mse}')
print(f'R2 Score: {r2}')

In [ ]:
#Step 7: Model Tuning and Optimization

#Hyperparameter Tuning:
#Use Grid Search or Random Search for hyperparameter tuning.
#Optimize parameters like the number of trees, depth of trees, learning rate, etc.

#Ensemble Methods:
#Combine multiple models to improve prediction accuracy (e.g., stacking, bagging).

# XGBoost # Define the hyperparameters grid
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.05, 0.1],
    'subsample': [0.7, 0.8, 0.9],
}

# Setup the GridSearchCV
grid_search = GridSearchCV(estimator=XGBRegressor(objective='reg:squarederror'), param_grid=param_grid, cv=3, scoring='neg_mean_squared_error', verbose=2, n_jobs=-1)

# Fit the model
grid_search.fit(X_train, y_train)

# Get the best parameters and score
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_
best_mse = -grid_search.best_score_

# Predict and evaluate
y_pred = best_model.predict(X_test)
best_r2 = r2_score(y_test, y_pred)
print(f'Best Model Mean Squared Error: {best_mse}')
print(f'Best Model R2 score: {best_r2}')
print(f'Best Parameters: {best_params}')
